# Deep Dive into Stacking Model for Multi-Class Classification

## Overview
This notebook demonstrates how to implement a stacking model for multi-class classification using configurations for multiple base models and a meta-model (final estimator). The stacking approach combines predictions from multiple base models to improve performance.


## Step 1: Import Libraries

In [6]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


## Step 2: Load Wine Quality Data

In [2]:
# Load the wine quality dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url, sep=';')

# Convert target to multi-class (low, medium, high quality)
def quality_label(q):
    if q <= 5:
        return 'low'
    elif q == 6:
        return 'medium'
    else:
        return 'high'

data['quality_label'] = data['quality'].apply(quality_label)
data.drop(columns=['quality'], inplace=True)

# Encode target labels
label_encoder = LabelEncoder()
data['quality_label'] = label_encoder.fit_transform(data['quality_label'])

# Features and target
X = data.drop(columns=['quality_label'])
y = data['quality_label']

## Step 3: Define Model Configurations

In [9]:
model_configs = {
    'model_1': {
        'feature_names': ['fixed acidity', 'volatile acidity'],
        'hyperparameters': {'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100},
        'estimators': LogisticRegression
    },
    'model_2': {
        'feature_names': ['citric acid', 'residual sugar'],
        'hyperparameters': {'n_estimators': 200, 'learning_rate': 0.05, 'verbosity': -1},
        'estimators': lgb.LGBMClassifier
    },
    'model_3': {
        'feature_names': ['chlorides', 'free sulfur dioxide'],
        'hyperparameters': None,
        'estimators': None
    }
}

final_estimator = lgb.LGBMClassifier(n_estimators=50, learning_rate=0.1, verbosity = -1)

## Step 4: Define the Stack Model Class

In [69]:
class StackModel:
    def __init__(self, model_configs, final_estimator):
        """
        Initialize the stacking model.

        Args:
            model_configs (dict): Configuration for base models.
            final_estimator (BaseEstimator): Meta-model for stacking.
        """
        self.model_configs = model_configs
        self.final_estimator = final_estimator
        self.models = {}

    def fit(self, X, y):
        """
        Train the stacking model.

        Args:
            X (pd.DataFrame): Feature matrix.
            y (pd.Series): Target vector.
        """
        self.models = {}
        self.meta_features = []

        for model_name, config in self.model_configs.items():
            features = config['feature_names']
            if config['estimators'] is not None:
                estimator = config['estimators'](**config['hyperparameters'])
                estimator.fit(X[features], y)

                self.models[model_name] = {
                    'features': features,
                    'model': estimator
                }

                # Generate meta-features using predictions
                meta_feature = estimator.predict_proba(X[features])
                self.meta_features.append(meta_feature)
            else:
                # Directly use the feature data for models without estimators
                self.models[model_name] = {
                    'features': features,
                    'model': None
                }
                self.meta_features.append(X[features].values)

        self.meta_features = np.hstack(self.meta_features)
        self.final_estimator.fit(self.meta_features, y)

    def predict(self, X):
        """
        Predict class labels using the stacking model.

        Args:
            X (pd.DataFrame): Feature matrix.

        Returns:
            np.ndarray: Predicted class labels.
        """
        meta_features = []

        for model_name, model_info in self.models.items():
            features = model_info['features']
            if model_info['model'] is not None:
                model = model_info['model']
                meta_features.append(model.predict_proba(X[features]))
            else:
                meta_features.append(X[features].values)                

        meta_features = np.hstack(meta_features)
        return self.final_estimator.predict(meta_features)

    def predict_proba(self, X):
        """
        Predict probabilities using the stacking model.

        Args:
            X (pd.DataFrame): Feature matrix.

        Returns:
            np.ndarray: Predicted probabilities.
        """
        meta_features = []

        for model_name, model_info in self.models.items():
            features = model_info['features']
            if model_info['model'] is not None:
                model = model_info['model']
                meta_features.append(model.predict_proba(X[features]))
            else:
                meta_features.append(X[features].values)                

        meta_features = np.hstack(meta_features)
        return self.final_estimator.predict_proba(meta_features)

    def generate_meta_features(self, X):
        """
        Generate meta-features for a given dataset.

        Args:
            X (pd.DataFrame): Feature matrix.

        Returns:
            pd.DataFrame: Meta-features generated by the base models.
        """
        meta_features = []
        columns = []

        for model_name, model_info in self.models.items():
            features = model_info['features']
            if model_info['model'] is not None:
                model = model_info['model']
                model_proba = model.predict_proba(X[features])
                meta_features.append(model_proba)
                n_classes = model_proba.shape[1]
                columns.extend([f"{model_name}_class{i}" for i in range(n_classes)])
            else:
                # Use raw features directly for models without estimators
                meta_features.append(X[features].values)
                columns.extend(features)

        meta_features = np.hstack(meta_features)
        return pd.DataFrame(meta_features, columns=columns)

## Step 5: Train and Evaluate the Stack Model

In [70]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the model
stack_model = StackModel(model_configs, final_estimator)
stack_model.fit(X_train, y_train)

# Predict and evaluate
predictions = stack_model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Accuracy of the stacking model: {accuracy:.4f}")

Accuracy of the stacking model: 0.5729


In [71]:
# Export meta-features for inspection
meta_features_train = stack_model.generate_meta_features(X_train)
meta_features_test = stack_model.generate_meta_features(X_test)
meta_features_train

,model_1_class0,model_1_class1,model_1_class2,model_2_class0,model_2_class1,model_2_class2,chlorides,free sulfur dioxide
0,0.324457,0.215730,0.459813,0.411349,0.134452,0.454199,0.064,53.0
1,0.185403,0.419977,0.394620,0.036433,0.646075,0.317492,0.071,6.0
2,0.102198,0.483839,0.413963,0.000267,0.861533,0.138200,0.084,12.0
3,0.055714,0.592318,0.351968,0.004361,0.420008,0.575631,0.045,19.0
4,0.088008,0.520087,0.391905,0.086474,0.548249,0.365277,0.077,27.0
...,...,...,...,...,...,...,...,...
1114,0.088350,0.542695,0.368955,0.010300,0.650717,0.338983,0.058,5.0
1115,0.072132,0.569988,0.357881,0.008834,0.430833,0.560332,0.073,25.0
1116,0.072850,0.553664,0.373486,0.019392,0.769255,0.211352,0.077,15.0
1117,0.330940,0.202417,0.466644,0.542588,0.052167,0.405246,0.054,7.0


## Compare the results of StackingClassifier (from sklearn) and the custom StackModel

In [47]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier

# Define feature selectors for each model
model_1_features = model_configs['model_1']['feature_names']
model_2_features = model_configs['model_2']['feature_names']
model_3_features = model_configs['model_3']['feature_names']

# Create pipelines for each base model
model_1_pipeline = Pipeline([
    ('selector', ColumnTransformer([('select', 'passthrough', model_1_features)])),
    ('model', LogisticRegression(**model_configs['model_1']['hyperparameters']))
])

model_2_pipeline = Pipeline([
    ('selector', ColumnTransformer([('select', 'passthrough', model_2_features)])),
    ('model', lgb.LGBMClassifier(**model_configs['model_2']['hyperparameters']))
])

model_3_pipeline = Pipeline([
    ('selector', ColumnTransformer([('select', 'passthrough', model_3_features)])),
    ('dummy', DummyClassifier(strategy='uniform'))  # Add a dummy classifier
])

# Define base models
base_models = [
    ('model_1', model_1_pipeline),
    ('model_2', model_2_pipeline),
    ('model_3', model_3_pipeline)
]

# Define the stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=final_estimator,
    passthrough=False
)

In [40]:
from sklearn.metrics import classification_report

# Custom StackModel
stack_model = StackModel(model_configs, final_estimator)
stack_model.fit(X_train, y_train)

# Predict and evaluate using the custom StackModel
custom_predictions = stack_model.predict(X_test)
custom_accuracy = accuracy_score(y_test, custom_predictions)
custom_report = classification_report(y_test, custom_predictions, target_names=label_encoder.classes_)

print("Custom StackModel Results:")
print(f"Accuracy: {custom_accuracy:.4f}")
print("Classification Report:")
print(custom_report)

# Sklearn StackingClassifier
stacking_clf.fit(X_train, y_train)

# Predict and evaluate using the sklearn StackingClassifier
sklearn_predictions = stacking_clf.predict(X_test)
sklearn_accuracy = accuracy_score(y_test, sklearn_predictions)
sklearn_report = classification_report(y_test, sklearn_predictions, target_names=label_encoder.classes_)

print("\nSklearn StackingClassifier Results:")
print(f"Accuracy: {sklearn_accuracy:.4f}")
print("Classification Report:")
print(sklearn_report)

# Compare Results
comparison = {
    "Metric": ["Accuracy"],
    "Custom StackModel": [custom_accuracy],
    "Sklearn StackingClassifier": [sklearn_accuracy]
}
comparison_df = pd.DataFrame(comparison)
print("\nComparison of Results:")
print(comparison_df)


Custom StackModel Results:
Accuracy: 0.5729
Classification Report:
              precision    recall  f1-score   support

        high       0.45      0.37      0.41        67
         low       0.64      0.65      0.65       213
      medium       0.54      0.56      0.55       200

    accuracy                           0.57       480
   macro avg       0.54      0.53      0.53       480
weighted avg       0.57      0.57      0.57       480


Sklearn StackingClassifier Results:
Accuracy: 0.5229
Classification Report:
              precision    recall  f1-score   support

        high       0.50      0.34      0.41        67
         low       0.56      0.64      0.60       213
      medium       0.48      0.46      0.47       200

    accuracy                           0.52       480
   macro avg       0.51      0.48      0.49       480
weighted avg       0.52      0.52      0.52       480


Comparison of Results:
     Metric  Custom StackModel  Sklearn StackingClassifier
0  Accuracy

## Make the results from StackModel and StackingClassifier

In [66]:
# Train both models
stack_model.fit(X_train, y_train)
stacking_clf.fit(X_train, y_train)

# Predictions
custom_predictions = stack_model.predict(X_test)
sklearn_predictions = stacking_clf.predict(X_test)

# Probabilities
custom_probabilities = stack_model.predict_proba(X_test)
sklearn_probabilities = stacking_clf.predict_proba(X_test)

# Accuracy comparison
print(f"Custom StackModel Accuracy: {accuracy_score(y_test, custom_predictions):.4f}")
print(f"Sklearn StackingClassifier Accuracy: {accuracy_score(y_test, sklearn_predictions):.4f}")

# Probabilities comparison
probability_difference = np.abs(custom_probabilities - sklearn_probabilities).max()
print(f"Max Difference in Probabilities: {probability_difference:.4e}")


Custom StackModel Accuracy: 0.5729
Sklearn StackingClassifier Accuracy: 0.5229
Max Difference in Probabilities: 9.0407e-01


In [95]:
import pandas as pd
import numpy as np

def generate_meta_features_clf(stacking_clf, X):
    """
    Generate meta-features from the base models in a stacking classifier.

    Args:
        stacking_clf: A fitted StackingClassifier instance.
        X (pd.DataFrame or np.ndarray): The input feature matrix.

    Returns:
        pd.DataFrame: A DataFrame of meta-features generated by base models.
    """
    meta_features = []
    columns = []

    for name, model in stacking_clf.named_estimators_.items():
        if hasattr(model, 'predict_proba'):  # Probability-based model
            proba = model.predict_proba(X)
            meta_features.append(proba)
            columns.extend([f"{name}_class_{i}" for i in range(proba.shape[1])])
        elif hasattr(model, 'decision_function'):  # Decision function-based model
            decision_scores = model.decision_function(X)
            if len(decision_scores.shape) == 1:  # Binary classification case
                decision_scores = decision_scores.reshape(-1, 1)
            meta_features.append(decision_scores)
            columns.append(f"{name}_decision")
        elif hasattr(model, 'predict'):  # Regressor or non-probability classifier
            preds = model.predict(X).reshape(-1, 1)
            meta_features.append(preds)
            columns.append(f"{name}_pred")

    # Stack the meta-features and convert to a DataFrame
    meta_features_array = np.hstack(meta_features)
    return pd.DataFrame(meta_features_array, columns=columns)


In [92]:
proba_train_clf = generate_meta_features_clf(stacking_clf, X_train)
proba_test_clf = generate_meta_features_clf(stacking_clf, X_test)

#### So sánh đầu vào của final estimator

In [93]:
proba_train_clf.head()

,model_1_class_0,model_1_class_1,model_1_class_2,model_2_class_0,model_2_class_1,model_2_class_2,model_3_class_0,model_3_class_1,model_3_class_2
0,0.324457,0.215730,0.459813,0.411349,0.134452,0.454199,0.333333,0.333333,0.333333
1,0.185403,0.419977,0.394620,0.036433,0.646075,0.317492,0.333333,0.333333,0.333333
2,0.102198,0.483839,0.413963,0.000267,0.861533,0.138200,0.333333,0.333333,0.333333
3,0.055714,0.592318,0.351968,0.004361,0.420008,0.575631,0.333333,0.333333,0.333333
4,0.088008,0.520087,0.391905,0.086474,0.548249,0.365277,0.333333,0.333333,0.333333


In [94]:
meta_features_train.head()

,model_1_class0,model_1_class1,model_1_class2,model_2_class0,model_2_class1,model_2_class2,chlorides,free sulfur dioxide
0,0.324457,0.215730,0.459813,0.411349,0.134452,0.454199,0.064,53.0
1,0.185403,0.419977,0.394620,0.036433,0.646075,0.317492,0.071,6.0
2,0.102198,0.483839,0.413963,0.000267,0.861533,0.138200,0.084,12.0
3,0.055714,0.592318,0.351968,0.004361,0.420008,0.575631,0.045,19.0
4,0.088008,0.520087,0.391905,0.086474,0.548249,0.365277,0.077,27.0


In [96]:
proba_test_clf.head()

,model_1_class_0,model_1_class_1,model_1_class_2,model_2_class_0,model_2_class_1,model_2_class_2,model_3_class_0,model_3_class_1,model_3_class_2
0,0.096785,0.503089,0.400127,0.404486,0.374178,0.221336,0.333333,0.333333,0.333333
1,0.123998,0.449699,0.426303,0.075519,0.901799,0.022682,0.333333,0.333333,0.333333
2,0.070821,0.607823,0.321356,0.001647,0.461695,0.536658,0.333333,0.333333,0.333333
3,0.150150,0.415067,0.434782,0.047777,0.138113,0.814110,0.333333,0.333333,0.333333
4,0.136066,0.411397,0.452537,0.006221,0.597996,0.395783,0.333333,0.333333,0.333333


In [97]:
meta_features_test.head()

,model_1_class0,model_1_class1,model_1_class2,model_2_class0,model_2_class1,model_2_class2,chlorides,free sulfur dioxide
0,0.096785,0.503089,0.400127,0.404486,0.374178,0.221336,0.114,14.0
1,0.123998,0.449699,0.426303,0.075519,0.901799,0.022682,0.082,21.0
2,0.070821,0.607823,0.321356,0.001647,0.461695,0.536658,0.107,17.0
3,0.150150,0.415067,0.434782,0.047777,0.138113,0.814110,0.078,32.0
4,0.136066,0.411397,0.452537,0.006221,0.597996,0.395783,0.077,18.0


#### Sử dụng riêng final estimator để kiểm tra kết quả

In [82]:
from sklearn.metrics import accuracy_score
import numpy as np
from copy import deepcopy
# Copy the final estimator for independent use
final_estimator_custom = deepcopy(final_estimator)  # For StackModel
final_estimator_clf = deepcopy(final_estimator)     # For StackingClassifier

# Train final estimators on respective meta-feature matrices
final_estimator_custom.fit(meta_features_train, y_train)  # For StackModel
final_estimator_clf.fit(proba_train_clf, y_train)         # For StackingClassifier

# Predictions
custom_predictions = final_estimator_custom.predict(meta_features_test)  # StackModel predictions
sklearn_predictions = final_estimator_clf.predict(proba_test_clf)        # StackingClassifier predictions

# Probabilities
custom_probabilities = final_estimator_custom.predict_proba(meta_features_test)  # StackModel probabilities
sklearn_probabilities = final_estimator_clf.predict_proba(proba_test_clf)        # StackingClassifier probabilities

# Accuracy comparison
custom_accuracy = accuracy_score(y_test, custom_predictions)
sklearn_accuracy = accuracy_score(y_test, sklearn_predictions)

print(f"Custom StackModel Accuracy: {custom_accuracy:.4f}")
print(f"Sklearn StackingClassifier Accuracy: {sklearn_accuracy:.4f}")

# Probabilities comparison
probability_difference = np.abs(custom_probabilities - sklearn_probabilities).max()
print(f"Max Difference in Probabilities: {probability_difference:.4e}")

# Additional check for consistency
if np.allclose(custom_probabilities, sklearn_probabilities, atol=1e-6):
    print("The custom StackModel and StackingClassifier produce nearly identical probabilities.")
else:
    print("The custom StackModel and StackingClassifier probabilities differ.")


Custom StackModel Accuracy: 0.5729
Sklearn StackingClassifier Accuracy: 0.5542
Max Difference in Probabilities: 6.3637e-01
The custom StackModel and StackingClassifier probabilities differ.


#### Sử dụng chung train and test data để so sánh

- kết quả ra giống nhau

In [102]:
from sklearn.metrics import accuracy_score
import numpy as np
from copy import deepcopy
# Copy the final estimator for independent use
final_estimator_custom = deepcopy(final_estimator)  # For StackModel
final_estimator_clf = deepcopy(final_estimator)     # For StackingClassifier

# Train final estimators on respective meta-feature matrices
final_estimator_custom.fit(meta_features_train, y_train)  # For StackModel
final_estimator_clf.fit(meta_features_train, y_train)         # For StackingClassifier

# Predictions
custom_predictions = final_estimator_custom.predict(meta_features_test)  # StackModel predictions
sklearn_predictions = final_estimator_clf.predict(meta_features_test)        # StackingClassifier predictions

# Probabilities
custom_probabilities = final_estimator_custom.predict_proba(meta_features_test)  # StackModel probabilities
sklearn_probabilities = final_estimator_clf.predict_proba(meta_features_test)        # StackingClassifier probabilities

# Accuracy comparison
custom_accuracy = accuracy_score(y_test, custom_predictions)
sklearn_accuracy = accuracy_score(y_test, sklearn_predictions)

print(f"Custom StackModel Accuracy: {custom_accuracy:.4f}")
print(f"Sklearn StackingClassifier Accuracy: {sklearn_accuracy:.4f}")

# Probabilities comparison
probability_difference = np.abs(custom_probabilities - sklearn_probabilities).max()
print(f"Max Difference in Probabilities: {probability_difference:.4e}")

# Additional check for consistency
if np.allclose(custom_probabilities, sklearn_probabilities, atol=1e-6):
    print("The custom StackModel and StackingClassifier produce nearly identical probabilities.")
else:
    print("The custom StackModel and StackingClassifier probabilities differ.")


Custom StackModel Accuracy: 0.5729
Sklearn StackingClassifier Accuracy: 0.5729
Max Difference in Probabilities: 0.0000e+00
The custom StackModel and StackingClassifier produce nearly identical probabilities.
